# ref

@kipf2016semi: <https://arxiv.org/abs/1609.02907>

In [2]:
import torch
import torch_geometric

# layer-wise progagation rule

@kipf2016semi use following layer-wise propagation rule: 

$$H^{(l+1)} = \sigma\big(\tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}H^{(l)}W^{(l)}\big).$$

Check followings: 

- In @kipf2016semi research, they suppose ${\cal G}$ as undirected graph. 
- $\tilde{A}=A+I_{N}$.
- $\tilde{D}_{ii}=\sum_{j}\tilde{A}_{ij}$.
- $W^{(l)}$ is a trainable weight matrix in $l$-th layer.
- $H^{(l)}$ is output of $l$-th layer; $H^{(0)}=X$. 

## 예제

`-` data 

In [3]:
x = torch.tensor([[20],
                  [21],
                  [19],
                  [1],
                  [2],
                  [1]], dtype=torch.float)

edge_index = torch.tensor([[0, 1, 2, 0, 1, 2, 3, 4, 3, 5, 4, 5],
                           [1, 0, 0, 2, 2, 1, 4, 3, 5, 3, 5, 4]], dtype=torch.long)

y = torch.tensor([1,1,1,0,0,0], dtype=torch.int64)
data = torch_geometric.data.Data(x=x, edge_index=edge_index, y=y) 
#data.train_mask = torch.tensor([True,False,True,True,False,True])
#data.test_mask = torch.tensor([False,True,False,False,True,False])
data

Data(x=[6, 1], edge_index=[2, 12], y=[6])

`-` GCNConv

In [4]:
gconv = torch_geometric.nn.GCNConv(1,4)
gconv

GCNConv(1, 4)

In [5]:
gconv(data.x, data.edge_index)

tensor([[ 11.6402, -15.0337, -13.0234, -15.7613],
        [ 11.6402, -15.0337, -13.0234, -15.7613],
        [ 11.6402, -15.0337, -13.0234, -15.7613],
        [  0.7760,  -1.0022,  -0.8682,  -1.0508],
        [  0.7760,  -1.0022,  -0.8682,  -1.0508],
        [  0.7760,  -1.0022,  -0.8682,  -1.0508]], grad_fn=<AddBackward0>)

In [12]:
list(gconv.parameters())

[Parameter containing:
 tensor([0., 0., 0., 0.], requires_grad=True),
 Parameter containing:
 tensor([[ 0.5820],
         [-0.7517],
         [-0.6512],
         [-0.7881]], requires_grad=True)]

In [15]:
_,W = list(gconv.parameters())
W

Parameter containing:
tensor([[ 0.5820],
        [-0.7517],
        [-0.6512],
        [-0.7881]], requires_grad=True)

In [16]:
A = torch.tensor([[0., 1., 1., 0., 0., 0.],
                  [1., 0., 1., 0., 0., 0.],
                  [1., 1., 0., 0., 0., 0.],
                  [0., 0., 0., 0., 1., 1.],
                  [0., 0., 0., 1., 0., 1.],
                  [0., 0., 0., 1., 1., 0.]])
Atilde = A+torch.eye(6)
Atilde

tensor([[1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.]])

In [17]:
Atilde@data.x@W.T/3, gconv(data.x,data.edge_index)

(tensor([[ 11.6402, -15.0337, -13.0234, -15.7613],
         [ 11.6402, -15.0337, -13.0234, -15.7613],
         [ 11.6402, -15.0337, -13.0234, -15.7613],
         [  0.7760,  -1.0022,  -0.8682,  -1.0508],
         [  0.7760,  -1.0022,  -0.8682,  -1.0508],
         [  0.7760,  -1.0022,  -0.8682,  -1.0508]], grad_fn=<DivBackward0>),
 tensor([[ 11.6402, -15.0337, -13.0234, -15.7613],
         [ 11.6402, -15.0337, -13.0234, -15.7613],
         [ 11.6402, -15.0337, -13.0234, -15.7613],
         [  0.7760,  -1.0022,  -0.8682,  -1.0508],
         [  0.7760,  -1.0022,  -0.8682,  -1.0508],
         [  0.7760,  -1.0022,  -0.8682,  -1.0508]], grad_fn=<AddBackward0>))

`-` 즉 아래의 수식에서 

$$H^{(l+1)} = \sigma\big(\tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}H^{(l)}W^{(l)}\big).$$

$\tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}H^{(l)}W^{(l)}$를 계산하는 Layer가 `torch_geometric.nn.GCNConv()` 으로 구현되어있음. 

# Spectral graph convolutions (Section 2.1-2)

`-` In this chapter, @kipf2016semi argues why the calculation 

$$H^{(l+1)} = \sigma\big(\tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}H^{(l)}W^{(l)}\big).$$

can be considered as graph convolution. 

`-` The properties of convolution operator $\star$ in classical spectral analysis are as follows (3):

$$g_{\theta} \star x = Ug_{\theta}U^\top x$$

where $g_{\theta}=\text{diag}(\theta)$ and $U$ is eigenvector matrix of $L=I-D^{-1/2}AD^{-1/2}$. 

`-` Now let's examine equation (7):

$$g_{\theta}\star x \approx \theta \big(I+D^{-1/2}AD^{-1/2}\big)x. \cdots (7)$$

By expressing equation (7) matrix form, we get equation (8): 

$$Z = \tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}X \Theta. \cdots (8)$$

This can be transfromed to 

$$H^{(l+1)} = \sigma\big(Z\big)= \sigma\big(\tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}H^{(l)}W^{(l)}\big).$$

where we interpret $H^{(l)}$ as $X$ and $W^{(l)}$ as $\Theta$. 